# Object Detection in video

This code is based on 
[zszazi's work](https://github.com/zszazi/Object-detection-in-video)

Clone the Tensorflow's repository

In [ ]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

Cloning into 'models'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 54665 (delta 19), reused 38 (delta 2), pack-reused 54603
Receiving objects: 100% (54665/54665), 570.03 MiB | 29.08 MiB/s, done.
Resolving deltas: 100% (37596/37596), done.


Install packages

In [ ]:
pip install tensorflow-object-detection-api

Clone the repository to import the models test videos

In [ ]:
%cd /content
import os
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename('https://github.com/Sheedy21/custom_object_detection.git')))

!git clone https://github.com/Sheedy21/custom_object_detection.git
%cd {repo_dir_path}
!git pull

/content
fatal: destination path 'custom_object_detection' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/custom_object_detection.git'
/content
fatal: not a git repository (or any of the parent directories): .git


Install packages

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from scipy.stats import itemfreq

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import tensorflow.compat.v1 as tf

Extract the models and label map files

In [ ]:
for i in range(1,4):
  locals()["PATH_TO_CKPT_" + str(i)] = '/content/custom_object_detection/Model_' + str(i) + '/frozen_inference_graph.pb'
  locals()["PATH_TO_LABELS_" + str(i)] = '/content/custom_object_detection/Model_' + str(i) + '/label_map.pbtxt'

NUM_CLASSES_1 = 3
NUM_CLASSES_2 = 3
NUM_CLASSES_3 = 1

for i in range(1,4):
    os.chdir('/content/custom_object_detection/Model_' + str(i) )
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.compat.v1.GraphDef() 
      with tf.compat.v2.io.gfile.GFile(locals()["PATH_TO_CKPT_" + str(i)], 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
    locals()["label_map_" + str(i)] = label_map_util.load_labelmap(locals()["PATH_TO_LABELS_" + str(i)])
    locals()["categories_" + str(i)] = label_map_util.convert_label_map_to_categories(locals()["label_map_" + str(i)], max_num_classes=locals()["NUM_CLASSES_" + str(i)], use_display_name=True)
    locals()["category_index_" + str(i)] = label_map_util.create_category_index(locals()["categories_" + str(i)])

Define some functions to load images

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [ ]:
#!wget https://storage.googleapis.com/videostestingobjectdetection/video_demo_baby_dog.mp4 -O videos/video_demo_baby_dog.mp4

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Evaluate the models using the videos

In [ ]:
for i in range (1,4):
    os.chdir('/content/custom_object_detection/Model_' + str(i) )

    out = cv2.VideoWriter('output_test.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (1280,720))

    filename = 'test_video.mp4'
    cap = cv2.VideoCapture(filename)

    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            counter = 0
            while (True):
                ret, image_np = cap.read()
                
                counter += 1
                if ret:
                  h = image_np.shape[0]
                  w = image_np.shape[1]

                  if counter % 1 == 0:
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    scores = detection_graph.get_tensor_by_name('detection_scores:0')
                    classes = detection_graph.get_tensor_by_name('detection_classes:0')
                    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                    (boxes, scores, classes, num_detections) = sess.run(
                        [boxes, scores, classes, num_detections],
                        feed_dict={image_tensor: image_np_expanded})

                    vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                                        np.squeeze(boxes),
                                                                        np.squeeze(classes).astype(np.int32),
                                                                        np.squeeze(scores),
                                                                        locals()["category_index_" + str(i)],
                                                                        use_normalized_coordinates=True,
                                                                        line_thickness=0,
                                                                        min_score_thresh=0.3)

                  out.write(image_np)
                else:
                  break
    out.release()
    cap.release()
    cv2.destroyAllWindows()